In [6]:
import pandas as pd
df = pd.read_csv('C:/users/hoang/Desktop/Study/CAP4770/Project/USVideos/USVideos.csv',
                 names = ['ID', 'Trending_Date', 'title', 'channel_title','category_id','publish_time',
'tags',
'views',
'likes',
'dislikes',
'comment_count',
'thumbnail_link',
'comments_disabled',
'ratings_disabled',
'video_error_or_removed',
'description'])


In [9]:
#-some attributes:
#title, tags, views, likes dislike, comment_count

data_notNull = df[(df['title'] != '[none]' ) & (df['tags'] != '[none]' )& (df['views'] != '[none]' )& (df['likes'] != '[none]' )&(df['dislikes'] != '[none]' )&(df['comment_count'] != '[none]' )]

In [21]:
 import re
#get stopwords with nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
# common_words = [word for word in word_list if word not in stopwords.words('english')]
def getWords(col:str):
    tagsArray = data_notNull[col]
    tag2D = []
    tagsArray
    
    for tag in tagsArray:
        lineArray = tag.split('|')
        words = []
        for sentence in lineArray:
            words = words + re.split(r'\W+', sentence.strip())
           
        tag2D.append(words)
        
    for i in range(len(tag2D)):
        tag2D[i] = [u for u in tag2D[i] if u.lower() not in stopwords.words('english')] 
                
    return tag2D

# tag2D  

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hoang\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [13]:
import hashlib
#turn all the string in the 2D array into some hash number
def hashArray(myArray):
    result = []
    for words in myArray:
        hashArray = []
        for word in words:
            hashArray.append(abs(hash(word)))
        result.append(hashArray)
    return result


In [ ]:
getWords('tags')